In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from typing import Optional, Any, List, Union, TypeVar, Type, cast, Callable
import re
from statistics import mean 


# for each subsection, consider adding into the current memory object
# the leaves text/structure

class Part:
    def __init__(self, Number, Title, Chapters):
        self.Number = Number
        self.Title = Title
        self.Chapters = Chapters

class Chapter:
    def __init__(self, Number, Title, Sections):
        self.Number = Number
        self.Title = Title
        self.Sections= Sections

class SubSection:
    Title: str
    Number: int
    Url: str
    def __init__(self, Number, Title, Url):
        self.Number = Number
        self.Title = Title
        self.Url = Url

class Section:
    Title: str
    SubSections: Optional[List[SubSection]]
    def __init__(self, Title, SubSections):
        self.Title = Title
        self.SubSections = SubSections        

def contents(url, type=1):
    urls = []
    ids = []
    txt = ''
    parts = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="xml")
    for part in  soup.select('Body > Part'):
        txt = txt + " <p> <b>"+part.find('Number').text+" "+part.find('Title').text+"</b></p>"
        chapters=[]
        if(len(part.find_all('Chapter'))>0):
            for chapter in part.find_all('Chapter'):
              # keep chapter
                if(len(chapter.find_all('Pblock'))>0):
                    subsections=[]
                    sections=[]
                    for result in chapter.find_all('Pblock'):
                        #print(" "+result.find('Title').text)
                        subsections=[]
                        txt = txt + " <p> " +result.find('Title').text+"</p>"
                        for first in result.select('Pblock > P1group'):
                            ids.append(first.select('Pnumber')[0].text)
                            txt = txt + " <p> "+first.select('Pnumber')[0].text+". "+first.find('Title').text+"</p>"
                            p1=first.findAll("P1")
                            if p1[0].has_attr('DocumentURI'):
                                urls.append(p1[0]['DocumentURI']+"/data.xml")
                                subsections.append(SubSection(first.select('Pnumber')[0].text, first.find('Title').text, p1[0]['DocumentURI']+"/data.xml"))
                        sections.append(Section(result.find('Title').text, subsections))
                else:
                    subsections=[]
                    sections=[]
                    for first in chapter.select('P1group'):
                            txt = txt + " <p> " +first.select('Pnumber')[0].text+". "+first.find('Title').text+"</p>"
                            ids.append(first.select('Pnumber')[0].text)
                            p1=first.findAll("P1")
                            if p1[0].has_attr('DocumentURI'):
                                urls.append(p1[0]['DocumentURI']+"/data.xml")
                                subsections.append(SubSection(first.select('Pnumber')[0].text, first.find('Title').text, p1[0]['DocumentURI']+"/data.xml"))
                    sections.append(Section(result.find('Title').text, subsections))
                chapters.append(Chapter(chapter.find('Number').text, chapter.find('Title').text, sections))                
                
        else:
                sections=[]
                for result in part.find_all('Pblock'):
                    txt = txt + " <p> " +result.find('Title').text+"</p>"
                    # code replication - improve
                    subsections=[]
                    for first in result.select('Pblock > P1group'):
                            if(len(first.findAll('P1'))>0):
                                txt = txt + " <p> " +first.select('Pnumber')[0].text+". "+first.find('Title').text+"</p>"
                                ids.append(first.select('Pnumber')[0].text)
                                p1=first.findAll("P1")
                                if p1[0].has_attr('DocumentURI'):
                                    urls.append(p1[0]['DocumentURI']+"/data.xml")
                                    subsections.append(SubSection(" "+first.select('Pnumber')[0].text, first.find('Title').text, p1[0]['DocumentURI']+"/data.xml"))
                    sections.append(Section(result.find('Title').text, subsections))
                chapters.append(Chapter("", "***", sections))

        parts.append(Part(part.find('Number').text, part.find('Title').text, chapters))

    if type == 1:
     return parts
    else:
     return parts


year=2004
number=34

parts=contents('https://www.legislation.gov.uk/ukpga/'+str(year)+'/'+str(number)+'/data.xml')


In [2]:
year=2004; number=34

# Housing Act 2004, 34
parts=contents('https://www.legislation.gov.uk/ukpga/'+str(year)+'/'+str(number)+'/data.xml')

for x in parts:
    print(str(x.Number).replace('\n',''))
    for y in x.Chapters:
        print(str(y.Number).replace('\n',''))
        for z in y.Sections:
            print(' '+str(z.Title).replace('\n',''))
            for t in z.SubSections:
                print(' '+str(t.Number)+" "+str(t.Title).replace('\n',''))
                print(t.Url)

Part 1
Chapter 1
 New system for assessing housing conditions
 1 New system for assessing housing conditions and enforcing housing standards
http://www.legislation.gov.uk/ukpga/2004/34/section/1/data.xml
 2 Meaning of  “category 1 hazard” and  “category 2 hazard”
http://www.legislation.gov.uk/ukpga/2004/34/section/2/data.xml
 Procedure for assessing housing conditions
 3 Local housing authorities to review housing conditions in their districts
http://www.legislation.gov.uk/ukpga/2004/34/section/3/data.xml
 4 Inspections by local housing authorities to see whether category 1 or 2 hazards exist
http://www.legislation.gov.uk/ukpga/2004/34/section/4/data.xml
 Enforcement of housing standards
 5 Category 1 hazards: general duty to take enforcement action
http://www.legislation.gov.uk/ukpga/2004/34/section/5/data.xml
 6 Category 1 hazards: how duty under section 5 operates in certain cases
http://www.legislation.gov.uk/ukpga/2004/34/section/6/data.xml
 7 Category 2 hazards: powers to take en